### 1단계: 네트워크 인프라 구성

#### 1.1 On-premise VNET 생성

**Azure Portal:**

1.  Azure Portal에 로그인합니다.
2.  "Create a resource"를 클릭합니다. (예시 스크린샷: "+" 버튼 클릭)
3.  "Virtual network"을 검색하여 선택하고 "Create"를 클릭합니다. (예시 스크린샷: 검색창에 "Virtual network" 입력 후 선택)
4.  다음 정보를 입력하고 "Review + create"를 클릭합니다.
    -   Subscription: 사용 중인 Azure 구독을 선택합니다.
    -   Resource group: "secu-ai-workshop"을 선택합니다.
    -   Name: "onprem-vnet"
    -   Region: "Korea Central"
    -   IP Addresses 탭에서 Address space를 "192.168.0.0/16"으로 설정합니다.
    -   Subnets 탭에서 다음 서브넷을 추가합니다.
        -   onprem-subnet: 192.168.1.0/24
        -   GatewaySubnet: 192.168.0.0/24 (주의: GatewaySubnet 이름은 변경 불가)
5.  "Create"를 클릭하여 VNET 생성을 완료합니다.

In [ ]:
# 변수 설정
RESOURCE_GROUP="secu-ai-workshop"
LOCATION="koreacentral"
ONPREM_VNET_NAME="onprem-vnet"
ONPREM_ADDRESS_PREFIX="192.168.0.0/16"
ONPREM_SUBNET_NAME="onprem-subnet"
ONPREM_SUBNET_PREFIX="192.168.1.0/24"
GATEWAY_SUBNET_NAME="GatewaySubnet" # GatewaySubnet 이름은 고정
GATEWAY_SUBNET_PREFIX="192.168.0.0/24"

# On-premise VNET 생성
az network vnet create\
    --name $ONPREM_VNET_NAME\
    --resource-group $RESOURCE_GROUP\
    --location $LOCATION\
    --address-prefixes $ONPREM_ADDRESS_PREFIX\
    --subnet-name $ONPREM_SUBNET_NAME\
    --subnet-prefixes $ONPREM_SUBNET_PREFIX

# GatewaySubnet 생성
az network vnet subnet create\
    --name $GATEWAY_SUBNET_NAME\
    --resource-group $RESOURCE_GROUP\
    --vnet-name $ONPREM_VNET_NAME\
    --address-prefix $GATEWAY_SUBNET_PREFIX

#### 1.2 Hub VNET 생성

**Azure Portal:**

1.  Azure Portal에서 "Create a resource"를 클릭합니다.
2.  "Virtual network"을 검색하여 선택하고 "Create"를 클릭합니다.
3.  다음 정보를 입력하고 "Review + create"를 클릭합니다.
    -   Subscription: 사용 중인 Azure 구독을 선택합니다.
    -   Resource group: "secu-ai-workshop"을 선택합니다.
    -   Name: "hubvnet"
    -   Region: "Korea Central"
    -   IP Addresses 탭에서 Address space를 "10.0.0.0/16"으로 설정합니다.
    -   Subnets 탭에서 다음 서브넷을 추가합니다.
        -   GatewaySubnet: 10.0.0.0/24
        -   AzureFirewallSubnet: 10.0.1.0/26 (주의: AzureFirewallSubnet 이름은 변경 불가)
        -   dmz: 10.0.2.0/24
4.  "Create"를 클릭하여 VNET 생성을 완료합니다.

In [ ]:
# 변수 설정
HUB_VNET_NAME="hubvnet"
HUB_ADDRESS_PREFIX="10.0.0.0/16"
HUB_GATEWAY_SUBNET_PREFIX="10.0.0.0/24"
FIREWALL_SUBNET_NAME="AzureFirewallSubnet" # AzureFirewallSubnet 이름은 고정
FIREWALL_SUBNET_PREFIX="10.0.1.0/26"
DMZ_SUBNET_NAME="dmz"
DMZ_SUBNET_PREFIX="10.0.2.0/24"

# Hub VNET 생성
az network vnet create\
    --name $HUB_VNET_NAME\
    --resource-group $RESOURCE_GROUP\
    --location $LOCATION\
    --address-prefixes $HUB_ADDRESS_PREFIX\
    --subnet-name $GATEWAY_SUBNET_NAME\
    --subnet-prefixes $HUB_GATEWAY_SUBNET_PREFIX

# AzureFirewallSubnet 생성
az network vnet subnet create\
    --name $FIREWALL_SUBNET_NAME\
    --resource-group $RESOURCE_GROUP\
    --vnet-name $HUB_VNET_NAME\
    --address-prefix $FIREWALL_SUBNET_PREFIX

# dmz 서브넷 생성
az network vnet subnet create\
    --name $DMZ_SUBNET_NAME\
    --resource-group $RESOURCE_GROUP\
    --vnet-name $HUB_VNET_NAME\
    --address-prefix $DMZ_SUBNET_PREFIX

#### 1.3 Spoke1 VNET 생성

**Azure Portal:**

(Hub VNET 생성 과정과 유사하며, Subnet 구성만 다름)

In [ ]:
# 변수 설정
SPOKE1_VNET_NAME="spoke1vnet"
SPOKE1_ADDRESS_PREFIX="10.1.0.0/16"
SPOKE1_SUBNET_NAME="spoke1subnet"
SPOKE1_SUBNET_PREFIX="10.1.0.0/24"
SPOKE1_PE_SUBNET_NAME="privateendpoint"
SPOKE1_PE_SUBNET_PREFIX="10.1.1.0/24"

# Spoke1 VNET 생성
az network vnet create\
  --name $SPOKE1_VNET_NAME\
  --resource-group $RESOURCE_GROUP\
  --location $LOCATION\
  --address-prefixes $SPOKE1_ADDRESS_PREFIX\
  --subnet-name $SPOKE1_SUBNET_NAME\
  --subnet-prefixes $SPOKE1_SUBNET_PREFIX

# Spoke1 privateendpoint 서브넷 생성
az network vnet subnet create\
  --name $SPOKE1_PE_SUBNET_NAME\
  --resource-group $RESOURCE_GROUP\
  --vnet-name $SPOKE1_VNET_NAME\
  --address-prefix $SPOKE1_PE_SUBNET_PREFIX

#### 1.4 Spoke2 VNET 생성

**Azure Portal:**

(Hub VNET 생성 과정과 유사하며, Subnet 구성만 다름)

In [ ]:
# 변수 설정
SPOKE2_VNET_NAME="spoke2vnet"
SPOKE2_ADDRESS_PREFIX="10.2.0.0/16"
SPOKE2_SUBNET_NAME="spoke2subnet"
SPOKE2_SUBNET_PREFIX="10.2.0.0/24"

# Spoke2 VNET 생성
az network vnet create\
  --name $SPOKE2_VNET_NAME\
  --resource-group $RESOURCE_GROUP\
  --location $LOCATION\
  --address-prefixes $SPOKE2_ADDRESS_PREFIX\
  --subnet-name $SPOKE2_SUBNET_NAME\
  --subnet-prefixes $SPOKE2_SUBNET_PREFIX

#### 1.5 VNET Peering (Hub ↔ Spoke1, Spoke2)

**Azure Portal (Hub ↔ Spoke1):**

1.  Hub VNET ("hubvnet") 설정 페이지로 이동합니다.
2.  "Settings" 아래의 "Peerings"를 선택하고 "Add"를 클릭합니다.
3.  다음 정보를 입력합니다.
    -   This virtual network:

        -   Peering link name: hub-to-spoke1
        -   Traffic to remote virtual network: Allow
        -   Traffic forwarded from remote virtual network: Allow
        -   Virtual network gateway or Route Server: Use this virtual network's gateway or Route Server

    -   ^1^Remote virtual network:

        -   Peering link name: spoke1-to-hub
        -   Traffic to remote virtual network: Allow
        -   Traffic forwarded from remote virtual network: Allow
        -   Virtual network gateway or Route Server: Use th^2^e remote virtual network's gateway or Route Server
        -   Virtual network: spoke1vnet 선택

4.  "Add"를 클릭합니다.

In [ ]:
# Hub VNET에서 Spoke1 VNET으로 Peering 설정
az network vnet peering create\
    --name hub-to-spoke1\
    --resource-group $RESOURCE_GROUP\
    --vnet-name $HUB_VNET_NAME\
    --remote-vnet $SPOKE1_VNET_NAME\
    --allow-vnet-access\
    --allow-forwarded-traffic\
    --allow-gateway-transit

# Spoke1 VNET에서 Hub VNET으로 Peering 설정
az network vnet peering create\
    --name spoke1-to-hub\
    --resource-group $RESOURCE_GROUP\
    --vnet-name $SPOKE1_VNET_NAME\
    --remote-vnet $HUB_VNET_NAME\
    --allow-vnet-access\
    --allow-forwarded-traffic\
    --use-remote-gateways

**Azure Portal (Hub ↔ Spoke2) 및 AZ CLI (Hub ↔ Spoke2):**

Spoke1과 동일한 방식으로 Peering을 설정합니다. (VNET 이름만 Spoke2로 변경)

In [ ]:
# Hub VNET에서 Spoke2 VNET으로 Peering 설정
az network vnet peering create\
    --name hub-to-spoke2\
    --resource-group $RESOURCE_GROUP\
    --vnet-name $HUB_VNET_NAME\
    --remote-vnet $SPOKE2_VNET_NAME\
    --allow-vnet-access\
    --allow-forwarded-traffic\
    --allow-gateway-transit

# Spoke2 VNET에서 Hub VNET으로 Peering 설정
az network vnet peering create\
    --name spoke2-to-hub\
    --resource-group $RESOURCE_GROUP\
    --vnet-name $SPOKE2_VNET_NAME\
    --remote-vnet $HUB_VNET_NAME\
    --allow-vnet-access\
    --allow-forwarded-traffic\
    --use-remote-gateways

**VNET Peering 설명:**

VNET Peering을 통해 서로 다른 VNET 간에 트래픽을 라우팅할 수 있습니다. Hub VNET은 Spoke VNET의 게이트웨이 역할을 하므로 'allow-gateway-transit' 옵션을 사용하고, Spoke VNET에서는 'use-remote-gateways'를 사용하여 Hub VNET의 게이트웨이를 사용하도록 설정합니다.

#### 1.6 Site-to-Site VPN (On-premise ↔ Hub)

**Azure Portal:**

1.  **Virtual Network Gateway 생성 (Hub VNET):**

    -   "Create a resource"에서 "Virtual network gateway"를 검색하여 선택하고 "Create"를 클릭합니다.
    -   다음 정보를 입력합니다.
        -   Name: hub-vgw
        -   Region: Korea Central
        -   Gateway type: VPN
        -   VPN type: Route-based
        -   SKU: 적절한 SKU 선택 (예: VpnGw1)
        -   Generation: Generation1 (또는 적절한 세대 선택)
        -   Virtual network: hubvnet
        -   Gateway subnet address range: 10.0.0.0/24 (Hub VNET의 GatewaySubnet)
        -   Public IP address: 새로 생성 (예: hub-vgw-pip)
        -   Enable active-active mode: Disabled (필요에 따라 활성화)
        -   Configure BGP: Disabled (필요에 따라 활성화)
    -   "Review + create"를 클릭하고 "Create"를 클릭합니다.
2.  **Local Network Gateway 생성 (Hub VNET용):**

    -   "Create a resource"에서 "Local network gateway"를 검색하여 선택하고 "Create"를 클릭합니다.
    -   다음 정보를 입력합니다.
        -   Name: hub-lgw
        -   IP address: On-premise VPN 장치의 Public IP 주소 (실제 주소로 변경)
        -   Address space: On-premise VNET의 주소 공간 (192.168.0.0/16)
        -   Region: Korea Central
    -   "Review + create"를 클릭하고 "Create"를 클릭합니다.
3.  **Virtual Network Gateway 생성 (On-prem VNET):**

    -   "Create a resource"에서 "Virtual network gateway"를 검색하여 선택하고 "Create"를 클릭합니다.
    -   다음 정보를 입력합니다.
        -   Name: onprem-vgw
        -   Region: Korea Central
        -   Gateway type: VPN
        -   VPN type: Route-based
        -   SKU: 적절한 SKU 선택 (예: VpnGw1)
        -   Generation: Generation1 (또는 적절한 세대 선택)
        -   Virtual network: onprem-vnet
        -   Gateway subnet address range: 192.168.0.0/24 (Onprem VNET의 GatewaySubnet)
        -   Public IP address: 새로 생성 (예: onprem-vgw-pip)
        -   Enable active-active mode: Disabled (필요에 따라 활성화)
        -   Configure BGP: Disabled (필요에 따라 활성화)
    -   "Review + create"를 클릭하고 "Create"를 클릭합니다.
4.  **Local Network Gateway 생성 (Onprem VNET용):**

    -   "Create a resource"에서 "Local network gateway"를 검색하여 선택하고 "Create"를 클릭합니다.
    -   다음 정보를 입력합니다.
        -   Name: onprem-lgw
        -   IP address: Hub VNET VPN 장치의 Public IP 주소 (실제 주소로 변경)
        -   Address space: Hub VNET의 주소 공간 (10.0.0.0/16)
        -   Region: Korea Central
    -   "Review + create"를 클릭하고 "Create"를 클릭합니다.
5.  **연결 생성 (Hub VNET):**

    -   Hub VNET의 Virtual Network Gateway (hub-vgw) 페이지로 이동합니다.

-   "Connections"에서 "Add"를 클릭합니다.
    -   다음 정보를 입력합니다.
        -   Name: hub-to-onprem
        -   Connection type: Site-to-site (IPsec)
        -   Virtual network gateway: hub-vgw
        -   Local network gateway: onprem-lgw
        -   Shared key (PSK): 공유 키 입력 (On-premise VPN 장치와 동일해야 함)
        -   Enable BGP: Disabled (필요에 따라 활성화)
    -   "OK"를 클릭합니다.

6.  **연결 생성 (On-prem VNET):**
    -   Onprem VNET의 Virtual Network Gateway (onprem-vgw) 페이지로 이동합니다.
    -   "Connections"에서 "Add"를 클릭합니다.
    -   다음 정보를 입력합니다.
        -   Name: onprem-to-hub
        -   Connection type: Site-to-site (IPsec)
        -   Virtual network gateway: onprem-vgw
        -   Local network gateway: hub-lgw
        -   Shared key (PSK): 공유 키 입력 (Hub VNET VPN 장치와 동일해야 함)
        -   Enable BGP: Disabled (필요에 따라 활성화)
    -   "OK"를 클릭합니다.

In [ ]:
# Hub VNET용 Virtual Network Gateway 생성
az network vnet-gateway create\
    --name hub-vgw\
    --resource-group $RESOURCE_GROUP\
    --vnet $HUB_VNET_NAME\
    --gateway-type Vpn\
    --vpn-type RouteBased\
    --sku VpnGw1 \ # 필요에 따라 변경
    --public-ip-addresses hub-vgw-pip\
    --no-wait

# Hub VNET용 Local Network Gateway 생성
# 주의: 실제 On-premise VPN 장치의 Public IP 주소로 변경해야 합니다.
ONPREM_VPN_IP="<On-premise_VPN_Public_IP>"

az network local-gateway create\
    --name hub-lgw\
    --resource-group $RESOURCE_GROUP\
    --gateway-ip-address $ONPREM_VPN_IP\
    --local-address-prefixes $ONPREM_ADDRESS_PREFIX

# On-prem VNET용 Virtual Network Gateway 생성
az network vnet-gateway create\
    --name onprem-vgw\
    --resource-group $RESOURCE_GROUP\
    --vnet $ONPREM_VNET_NAME\
    --gateway-type Vpn\
    --vpn-type RouteBased\
    --sku VpnGw1 \ # 필요에 따라 변경
    --public-ip-addresses onprem-vgw-pip\
    --no-wait

# On-prem VNET용 Local Network Gateway 생성
# 주의: 실제 Hub VNET VPN 장치의 Public IP 주소는 az network public-ip show 명령어로 확인 후 넣어야 합니다.
HUB_VPN_PIP_NAME="hub-vgw-pip"
HUB_VPN_IP=$(az network public-ip show --name $HUB_VPN_PIP_NAME --resource-group $RESOURCE_GROUP --query ipAddress --output tsv)

az network local-gateway create\
    --name onprem-lgw\
    --resource-group $RESOURCE_GROUP\
    --gateway-ip-address $HUB_VPN_IP\
    --local-address-prefixes $HUB_ADDRESS_PREFIX

# Hub VNET에서 연결 생성
# 주의: 공유 키는 안전하게 보관하고, On-premise VPN 장치와 동일하게 설정해야 합니다.
SHARED_KEY="<Your_Shared_Key>"

az network vnet-gateway connection create\
    --name hub-to-onprem\
    --resource-group $RESOURCE_GROUP\
    --vnet-gateway1 hub-vgw\
    --shared-key $SHARED_KEY\
    --local-gateway2 onprem-lgw

# On-prem VNET에서 연결 생성
az network vnet-gateway connection create\
    --name onprem-to-hub\
    --resource-group $RESOURCE_GROUP\
    --vnet-gateway1 onprem-vgw\
    --shared-key $SHARED_KEY\
    --local-gateway2 hub-lgw

**Site-to-Site VPN 설명:**

Site-to-Site VPN은 On-premise 네트워크와 Azure VNET 간에 안전한 암호화된 연결을 제공합니다. Virtual Network Gateway는 VPN 연결의 엔드포인트 역할을 하며, Local Network Gateway는 상대방 네트워크(이 경우 On-premise 또는 Hub)를 나타냅니다.

#### 1.7 Azure Firewall (Hub VNET에 배치)

**Azure Portal:**

1.  "Create a resource"에서 "Firewall"을 검색하여 선택하고 "Create"를 클릭합니다.
2.  다음 정보를 입력합니다.
    -   Subscription: 사용 중인 Azure 구독을 선택합니다.
    -   Resource group: "secu-ai-workshop"을 선택합니다.
    -   Name: "hub-firewall"
    -   Region: "Korea Central"
    -   Firewall SKU: Standard (필요에 따라 선택)
    -   Firewall management: Use a Firewall Policy to manage this firewall
    -   Firewall policy: 새 정책 생성 (예: hub-firewall-policy)
    -   Choose a virtual network: Use existing: hubvnet
    -   Subnet: AzureFirewallSubnet (자동으로 선택됨)
    -   Public IP address: 새로 생성 (예: hub-firewall-pip)
3.  "Review + create"를 클릭하고 "Create"를 클릭합니다.

In [ ]:
# 변수 설정
FIREWALL_NAME="hub-firewall"
FIREWALL_POLICY_NAME="hub-firewall-policy"
FIREWALL_PIP_NAME="hub-firewall-pip"

# Azure Firewall Policy 생성
az network firewall policy create\
    --name $FIREWALL_POLICY_NAME\
    --resource-group $RESOURCE_GROUP\
    --location $LOCATION

# Azure Firewall Public IP 생성
az network public-ip create\
    --name $FIREWALL_PIP_NAME\
    --resource-group $RESOURCE_GROUP\
    --location $LOCATION\
    --sku Standard

# Azure Firewall 생성
az network firewall create\
    --name $FIREWALL_NAME\
    --resource-group $RESOURCE_GROUP\
    --location $LOCATION\
    --policy $FIREWALL_POLICY_NAME\
    --vnet-name $HUB_VNET_NAME\
    --public-ip-address $FIREWALL_PIP_NAME

#### 1.8 Azure Firewall Policy (Network Rule Collection)

**Azure Portal:**

1.  생성된 Firewall Policy (hub-firewall-policy) 페이지로 이동합니다.

2.  "Settings" 아래의 "Network Rule Collections"를 선택하고 "Add network rule collection"을 클릭합니다.

3.  다음 정보를 입력하여 규칙 컬렉션을 추가합니다.

    -   **Onprem to Spoke1:**
        -   Name: onprem-to-spoke1
        -   Priority: 100
        -   Action: Allow
        -   Rules:
            -   Name: allow-all
            -   Source type: IP Address
            -   Source addresses: 192.168.0.0/16
            -   Destination type: IP Address
            -   Destination addresses: 10.1.0.0/16
            -   Protocols: Any
            -   Ports: \*
    -   **Onprem to Spoke2:**
        -   Name: onprem-to-spoke2
        -   Priority: 101
        -   Action: Allow
        -   Rules:
        -   Name: allow-all
        -   Source type: IP Address
        -   Source addresses: 192.168.0.0/16
        -   Destination type: IP Address
        -   Destination addresses: 10.2.0.0/16
        -   Protocols: Any
        -   Ports: \*
    -   **Spoke1 to Spoke2:**
        -   Name: spoke1-to-spoke2
        -   Priority: 103
        -   Action: Allow
        -   Rules:
        -   Name: allow-all
        -   Source type: IP Address
        -   Source addresses: 10.1.0.0/16
        -   Destination type: IP Address
        -   Destination addresses: 10.2.0.0/16
        -   Protocols: Any
        -   Ports: \*
    -   **Spoke2 to Spoke1:**
        -   Name: spoke2-to-spoke1
        -   Priority: 104
        -   Action: Allow
        -   Rules:
        -   Name: allow-all
        -   Source type: IP Address
        -   Source addresses: 10.2.0.0/16
        -   Destination type: IP Address
        -   Destination addresses: 10.1.0.0/16
        -   Protocols: Any
        -   Ports: \*
4.  "Add"를 클릭하여 규칙 컬렉션을 저장합니다.

In [ ]:
# Firewall Policy ID 가져오기
FIREWALL_POLICY_ID=$(az network firewall policy show --name $FIREWALL_POLICY_NAME --resource-group $RESOURCE_GROUP --query id -o tsv)
# Firewall IP Address
FIREWALL_IP="10.0.1.4"

# Onprem to Spoke1 규칙 컬렉션 추가
az network firewall policy rule-collection-group collection add-filter-collection\
    --rule-collection-group-name DefaultNetworkRuleCollectionGroup\
    --policy-name $FIREWALL_POLICY_NAME\
    --resource-group $RESOURCE_GROUP\
    --name onprem-to-spoke1\
    --priority 100\
    --action Allow\
    --rule-type NetworkRule\
    --rules name=allow-all\
           source-addresses=192.168.0.0/16\
           destination-addresses=10.1.0.0/16\
           ip-protocols=Any\
           destination-ports='*'

# Onprem to Spoke2 규칙 컬렉션 추가
az network firewall policy rule-collection-group collection add-filter-collection\
    --rule-collection-group-name DefaultNetworkRuleCollectionGroup\
    --policy-name $FIREWALL_POLICY_NAME\
    --resource-group $RESOURCE_GROUP\
    --name onprem-to-spoke2\
    --priority 101\
    --action Allow\
    --rule-type NetworkRule\
    --rules name=allow-all\
           source-addresses=192.168.0.0/16\
           destination-addresses=10.2.0.0/16\
           ip-protocols=Any\
           destination-ports='*'

# Spoke1 to Spoke2 규칙 컬렉션 추가
az network firewall policy rule-collection-group collection add-filter-collection\
--rule-collection-group-name DefaultNetworkRuleCollectionGroup\
    --policy-name $FIREWALL_POLICY_NAME\
    --resource-group $RESOURCE_GROUP\
    --name spoke1-to-spoke2\
    --priority 103\
    --action Allow\
    --rule-type NetworkRule\
    --rules name=allow-all\
           source-addresses=10.1.0.0/16\
           destination-addresses=10.2.0.0/16\
           ip-protocols=Any\
           destination-ports='*'

# Spoke2 to Spoke1 규칙 컬렉션 추가
az network firewall policy rule-collection-group collection add-filter-collection\
--rule-collection-group-name DefaultNetworkRuleCollectionGroup\
    --policy-name $FIREWALL_POLICY_NAME\
    --resource-group $RESOURCE_GROUP\
    --name spoke2-to-spoke1\
    --priority 104\
    --action Allow\
    --rule-type NetworkRule\
    --rules name=allow-all\
           source-addresses=10.2.0.0/16\
           destination-addresses=10.1.0.0/16\
           ip-protocols=Any\
           destination-ports='*'

#### 1.9 Route Table

##### 1.9.1 Hub Route Table (Hub VNET GatewaySubnet에 연결)

**Azure Portal:**

1.  "Create a resource"에서 "Route table"을 검색하여 선택하고 "Create"를 클릭합니다.

2.  다음 정보를 입력합니다.

    -   Subscription: 사용 중인 Azure 구독
    -   Resource group: "secu-ai-workshop"
    -   Location: "Korea Central"
    -   Name: "hub-route-table"
    -   Propagate gateway routes: No
3.  "Review + create"를 클릭하고 "Create"를 클릭합니다.

4.  생성된 Route Table (hub-route-table) 페이지로 이동합니다.

5.  "Settings" 아래의 "Routes"를 선택하고 "Add"를 클릭합니다.

6.  다음 정보를 입력하여 경로를 추가합니다.

    -   Route name: to-spoke1
    -   Address prefix destination: IP Addresses
    -   Destination IP addresses/CIDR ranges: 10.1.0.0/16
    -   Next hop type: Virtual appliance
    -   Next hop address: 10.0.1.4 (Azure Firewall IP)
    -   Route name: to-spoke2
    -   Address prefix destination: IP Addresses
    -   Destination IP addresses/CIDR ranges: 10.2.0.0/16
    -   Next hop type: Virtual appliance
    -   Next hop address: 10.0.1.4 (Azure Firewall IP)
7.  "Settings" 아래 "Subnets"를 선택하고 "Associate"를 클릭합니다.

    -   Virtual network: hubvnet 선택
    -   Subnet: GatewaySubnet 선택
8.  "Associate"를 클릭합니다.

In [ ]:
# Route Table 생성
az network route-table create\
    --name hub-route-table\
    --resource-group $RESOURCE_GROUP\
    --location $LOCATION\
    --disable-bgp-route-propagation true

# Spoke1으로 가는 경로 추가
az network route-table route create\
    --name to-spoke1\
    --resource-group $RESOURCE_GROUP\
    --route-table-name hub-route-table\
    --address-prefix 10.1.0.0/16\
    --next-hop-type VirtualAppliance\
    --next-hop-ip-address $FIREWALL_IP

# Spoke2으로 가는 경로 추가
az network route-table route create\
    --name to-spoke2\
    --resource-group $RESOURCE_GROUP\
    --route-table-name hub-route-table\
    --address-prefix 10.2.0.0/16\
    --next-hop-type VirtualAppliance\
    --next-hop-ip-address $FIREWALL_IP

# Hub VNET GatewaySubnet에 연결
az network vnet subnet update\
    --name $GATEWAY_SUBNET_NAME\
    --resource-group $RESOURCE_GROUP\
    --vnet-name $HUB_VNET_NAME\
    --route-table hub-route-table


##### 1.9.2 Spoke1 Route Table (Spoke1 VNET에 연결)

**Azure Portal:**

(Hub Route Table과 유사하게 생성하고, 경로는 0.0.0.0/0 트래픽을 Firewall로 전달)

In [ ]:
# Route Table 생성
az network route-table create\
    --name spoke1-route-table\
    --resource-group $RESOURCE_GROUP\
    --location $LOCATION\
    --disable-bgp-route-propagation true

# 0.0.0.0/0 트래픽을 Firewall로 전달하는 경로 추가
az network route-table route create\
    --name to-firewall\
    --resource-group $RESOURCE_GROUP\
    --route-table-name spoke1-route-table\
    --address-prefix 0.0.0.0/0\
    --next-hop-type VirtualAppliance\
    --next-hop-ip-address $FIREWALL_IP

# Spoke1 VNET에 연결
az network vnet subnet update\
    --name $SPOKE1_SUBNET_NAME\
    --resource-group $RESOURCE_GROUP\
    --vnet-name $SPOKE1_VNET_NAME\
    --route-table spoke1-route-table

##### 1.9.3 Spoke2 Route Table (Spoke2 VNET에 연결)

**Azure Portal:**

(Spoke1 Route Table과 동일하게 생성)


In [ ]:
# Route Table 생성
az network route-table create\
    --name spoke2-route-table\
    --resource-group $RESOURCE_GROUP\
    --location $LOCATION\
    --disable-bgp-route-propagation true
# 0.0.0.0/0 트래픽을 Firewall로 전달하는 경로 추가
az network route-table route create\
    --name to-fire
    --resource-group $RESOURCE_GROUP\
    --route-table-name spoke2-route-table\
    --address-prefix 0.0.0.0/0\
    --next-hop-type VirtualAppliance\
    --next-hop-ip-address $FIREWALL_IP
# Spoke2 VNET에 연결
az network vnet subnet update\
    --name $SPOKE2_SUBNET_NAME\
    --resource-group $RESOURCE_GROUP\
    --vnet-name $SPOKE2_VNET_NAME\
    --route-table spoke2-route-table


**Route Table 설명:**

-   **Hub Route Table:** Spoke VNET으로 향하는 트래픽을 Azure Firewall로 전달하여 보안 정책을 적용합니다.
-   **Spoke Route Table:** 인터넷으로 나가는 트래픽 (0.0.0.0/0)을 Azure Firewall로 전달하여 중앙 집중식 보안 관리를 수행합니다.